In [1]:
from tomato.symbolic.symbtranalyzer import SymbTrAnalyzer
from tomato.metadata.work import Work
import os
from pprint import pprint


In [2]:
# symbtr name, 
# it is needed if the filename is modified from the SymbTr naming convention
symbtr_name = 'ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede'

# score filepaths
txt_filename = os.path.join(symbtr_name, symbtr_name + '.txt')
mu2_filename = os.path.join(symbtr_name, symbtr_name + '.mu2')

# instantiate analyzer object
scoreAnalyzer = SymbTrAnalyzer(verbose=True)


You can use the single line call "analyze," which does all the available analysis simultaneously

In [3]:
%pdb
score_data = scoreAnalyzer.analyze(
    txt_filename, None, symbtr_name=symbtr_name)


Automatic pdb calling has been turned ON
- Automatic phrase segmentation on the SymbTr-txt file: ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt
  The call took 9.71 seconds to execute.


In [4]:
# pretty print the metadata
pprint(score_data['metadata'])


{'composer': {'mbid': 'a6775723-69b6-4997-82da-bf230b1f8655',
              'name': 'Neyzen Aziz Dede',
              'symbtr_slug': 'neyzen_aziz_dede'},
 'duration': {'unit': 'second', 'value': 275.821},
 'form': {'attribute_key': 'sazsemaisi',
          'mb_attribute': 'Sazsemaisi',
          'source': 'http://musicbrainz.org/work/e7924b0d-c8a0-4b4a-b253-8eec898eac1e',
          'symbtr_slug': 'sazsemaisi'},
 'language': 'zxx',
 'lyricist': {},
 'makam': {'attribute_key': 'ussak',
           'mb_attribute': 'Uşşak',
           'source': 'http://musicbrainz.org/work/e7924b0d-c8a0-4b4a-b253-8eec898eac1e',
           'symbtr_slug': 'ussak'},
 'number_of_notes': 680,
 'recordings': [{'mbid': '86721a3a-268f-4422-8803-73a196d2a571',
                 'title': 'Uşşak Sazsemaisi'},
                {'mbid': '9a7d391f-90a6-4f2d-a0d2-5e6a5365b149',
                 'title': 'Uşşak Saz Semaisi'},
                {'mbid': '9b869a16-0ab0-43a2-a51e-2a03e9484e70',
                 'title': 'Uşşak Saz

... or you can call all the methods individually


In [5]:
# Automatic phrase segmentation on the SymbTr-txt score using pre-trained model
phrase_bounds = scoreAnalyzer.segment_phrase(txt_filename, symbtr_name=symbtr_name)

# relevant recording or work mbid, if you want additional information from musicbrainz
# Note 1: MBID input will make the function returns significantly slower because we
#         have to wait a couple of seconds before each subsequent query from MusicBrainz.
# Note 2: very rare but there can be more that one mbid returned. We are going to use 
#         the first work to get fetch the metadata
mbid = Work.get_mbids(symbtr_name)[0]

# Extract the (meta)data from the SymbTr scores
# This includes the metadata from the score formats and MusicBrainz, extracted sections
# rhythmic structure etc.
score_features, validity = scoreAnalyzer.extract_data(
    txt_filename, mu2_filename, symbtr_name=symbtr_name, mbid=mbid, 
    segment_note_bound_idx=phrase_bounds['boundary_note_idx'])


- Automatic phrase segmentation on the SymbTr-txt file: ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt
  The call took 8.93 seconds to execute.


AttributeError: type object 'Work' has no attribute 'get_mbids'

> <ipython-input-5-a97864e2a475>(9)<module>()
      7 # Note 2: very rare but there can be more that one mbid returned. We are going to use
      8 #         the first work to get fetch the metadata
----> 9 mbid = Work.get_mbids(symbtr_name)[0]
     10 
     11 # Extract the (meta)data from the SymbTr scores

ipdb> q
